<a href="https://colab.research.google.com/github/liliiakryvelova/build_book_recommender/blob/main/vector_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

ModuleNotFoundError: No module named 'langchain_community'

In [ ]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from dotenv import load_dotenv

load_dotenv()


True

In [ ]:
import pandas as pd
books = pd.read_csv("books_cleaned.csv")


In [5]:
# Explore the books dataset
print(f"Dataset shape: {books.shape}")
print(f"\nColumns: {list(books.columns)}")
print(f"\nFirst few rows:")
books.head()

Dataset shape: (6507, 12)

Columns: ['isbn13', 'isbn10', 'title', 'authors', 'categories', 'thumbnail', 'published_year', 'average_rating', 'num_pages', 'ratings_count', 'title_and_subtitle', 'tagged_description']

First few rows:


,isbn13,isbn10,title,authors,categories,thumbnail,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,2004.0,3.85,247.0,361.0,Gilead,9780002005883A NOVEL THAT READERS and critics ...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982A new 'Christie for Christmas' --...
2,9780006163831,0006163831,The One Tree,Stephen R. Donaldson,American fiction,http://books.google.com/books/content?id=OmQaw...,1982.0,3.97,479.0,172.0,The One Tree,9780006163831Volume Two of Stephen Donaldson's...
3,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736A memorable, mesmerizing heroine ..."
4,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897Lewis' work on the nature of love...


In [6]:
# Prepare text data for vector search
# Combine relevant text fields for each book
books['combined_text'] = (
    books['title'].fillna('') + ' ' + 
    books['authors'].fillna('') + ' ' + 
    books['categories'].fillna('') + ' ' + 
    books['tagged_description'].fillna('')
)

# Check for any missing data
print(f"Books with combined text: {len(books[books['combined_text'].str.strip() != ''])}")
print(f"Sample combined text:")
print(books['combined_text'].iloc[0][:200] + "...")

Books with combined text: 6507
Sample combined text:
Gilead Marilynne Robinson Fiction 9780002005883A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Am...


In [7]:
# Set up vector store and embeddings
import os

# Initialize OpenAI embeddings (make sure you have OPENAI_API_KEY in your .env file)
embeddings = OpenAIEmbeddings()

# Convert books data to documents for vector search
documents = []
for idx, row in books.iterrows():
    # Create a document with the combined text and metadata
    doc_text = row['combined_text']
    metadata = {
        'title': row['title'],
        'authors': row['authors'],
        'categories': row['categories'],
        'average_rating': row['average_rating'],
        'isbn13': row['isbn13']
    }
    documents.append({'text': doc_text, 'metadata': metadata})

print(f"Created {len(documents)} documents for vector search")
print(f"Sample document metadata: {documents[0]['metadata']}")

NameError: name 'OpenAIEmbeddings' is not defined